## Part 2: PCA Method with Maximum Likelihood Estimation (MLE)

---

**Name:** Arwa Ahmed Mostafa Shazly  
**ID:** 221100209  

---

In [23]:
import numpy as np
import pandas as pd
from scipy import linalg
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

## Load and Prepare Data

In [24]:
ratings_df = pd.read_csv('ratings.csv')
print(f"Original: {len(ratings_df):,} ratings")
orig_min, orig_max = ratings_df['rating'].min(), ratings_df['rating'].max()
ratings_df['rating'] = (ratings_df['rating'] - orig_min) / (orig_max - orig_min) * 4 + 1
print(f"Scale: 1 to 5")

Original: 20,000,263 ratings
Scale: 1 to 5


In [40]:
# Filter data (Random Sample)
user_counts = ratings_df['userId'].value_counts()
item_counts = ratings_df['movieId'].value_counts()

valid_items = item_counts[item_counts >= 20].index.tolist()
valid_users = user_counts[user_counts >= 20].index.tolist()

np.random.seed(42)
sample_items = set(np.random.choice(valid_items, size=800, replace=False))
sample_items.add(2)
sample_items.add(8860)
sample_users = set(np.random.choice(valid_users, size=15000, replace=False))

filtered_df = ratings_df[
    (ratings_df['userId'].isin(sample_users)) & 
    (ratings_df['movieId'].isin(sample_items))
]

print(f"Filtered sample: {len(filtered_df):,} ratings, {filtered_df['userId'].nunique():,} users, {filtered_df['movieId'].nunique()} items")

Filtered sample: 142,888 ratings, 14,201 users, 800 items


In [26]:
I1 = 2
I2 = 8860

def train_test_split_per_user(df, test_ratio=0.2, seed=42):
    np.random.seed(seed)
    train_list, test_list = [], []
    for uid, udata in df.groupby('userId'):
        n = len(udata)
        if n < 5:
            train_list.append(udata)
            continue
        shuffled = udata.sample(frac=1, random_state=seed)
        n_test = max(1, int(n * test_ratio))
        test_list.append(shuffled.iloc[:n_test])
        train_list.append(shuffled.iloc[n_test:])
    return pd.concat(train_list), pd.concat(test_list)

train_df, test_df = train_test_split_per_user(filtered_df)
rating_matrix = train_df.pivot_table(index='userId', columns='movieId', values='rating')
print(f"Rating Matrix: {rating_matrix.shape}")

Rating Matrix: (14201, 796)


---
# Point 1: MLE-based Covariance Matrix

- Step 1: Compute item means using only observed ratings
- Step 2: Center the data around item means
- Step 3: Form products of deviations
- Step 4: Estimate covariances using only co-rated entries

In [41]:
def compute_mle_covariance_matrix(rating_matrix):
    data = rating_matrix.values.astype(np.float32)
    items = rating_matrix.columns
    mask = ~np.isnan(data) 
    
    # STEP 1: Compute item means using only observed ratings
    # For each product, average only over users who have specified ratings
    means = np.nanmean(data, axis=0)
    
    # STEP 2: Center the data around item means
    # X_i = P1_i - mean(P1), Y_i = P2_i - mean(P2)
    centered = np.where(mask, data - means, 0).astype(np.float32)
    
    # STEP 3: Form products of deviations, (X_i)(Y_i) for each pair of items
    products_sum = centered.T @ centered
    
    # STEP 4: Divide by (n-1) co-rated users, cov(A,B) = Sum / (n_AB - 1)
    # Only users who rated BOTH items contribute
    co_rated_counts = mask.astype(np.float32).T @ mask.astype(np.float32)
    denominator = np.maximum(co_rated_counts - 1, 1)
    cov_matrix = np.divide(products_sum, denominator, 
                           out=np.zeros_like(products_sum), 
                           where=co_rated_counts > 1)
    
    return pd.DataFrame(cov_matrix, index=items, columns=items)

cov_matrix = compute_mle_covariance_matrix(rating_matrix)
print(f"\nCovariance matrix shape: {cov_matrix.shape}")
print(f"Symmetric: {np.allclose(cov_matrix.values, cov_matrix.values.T)} ")


Covariance matrix shape: (796, 796)
Symmetric: True 


---
# Point 2: Top Peers
Using the MLE covariance matrix, select items with highest covariance to target.

In [42]:
def get_top_peers(cov_matrix, target_item, k):
    return cov_matrix[target_item].drop(target_item).nlargest(k)

top5_I1 = get_top_peers(cov_matrix, I1, 5)
top5_I2 = get_top_peers(cov_matrix, I2, 5)
top10_I1 = get_top_peers(cov_matrix, I1, 10)
top10_I2 = get_top_peers(cov_matrix, I2, 10)

print(f"\nTop 5 peers for I1 (movieId={I1}):")
print(top5_I1.round(4))
print(f"\nTop 5 peers for I2 (movieId={I2}):")
print(top5_I2.round(4))


Top 5 peers for I1 (movieId=2):
movieId
26887    3.6066
106      2.0141
6956     1.4735
1922     1.4254
6177     1.2812
Name: 2, dtype: float32

Top 5 peers for I2 (movieId=8860):
movieId
6978      4.3112
581       4.2948
8650      3.5571
103688    3.3277
5815      2.8036
Name: 8860, dtype: float32


---
# Points 3-4: PCA with 5 Peers
1. Compute eigenvalues λ and eigenvectors p of Σ_MLE: det(Σ - λI) = 0
2. Center data and project users into latent space: t_i = x_i · p
3. Reconstruct missing ratings: r̂ = mean + t · p^T

In [43]:
def pca_predict_with_steps(rating_matrix, target_item, peers, var_thresh=0.9):
    items = [target_item] + list(peers.index)
    submatrix = rating_matrix[items]
    item_means = submatrix.mean()
    filled = submatrix.fillna(item_means)
    
    # PCA STEP 1: Center the data, X'_i = P1_i - mean(P1)
    centered = filled - item_means
    # Covariance of submatrix
    cov = np.cov(centered.T)
    
    # PCA STEP 2: Eigendecomposition
    # det(Σ - λI) = 0
    # Get eigenvalues (λ) and eigenvectors (p)
    eigenvalues, eigenvectors = linalg.eigh(cov)
    
    # Sort by eigenvalue (descending)
    idx = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    # PCA STEP 3: Select components (90% variance threshold)
    # Fraction of total variance = λ_k / Σλ
    cumvar = np.cumsum(eigenvalues) / eigenvalues.sum()
    n_comp = max(1, np.argmax(cumvar >= var_thresh) + 1)
    n_comp = min(n_comp, len(items))
    
    # Principal components (eigenvectors for selected components)
    pc = eigenvectors[:, :n_comp]
    
    # PCA STEP 4: Project users into latent space
    # t_i = x_i · p (user score on PC)
    # This gives the LATENT FACTORS
    latent = centered.values @ pc
    latent_df = pd.DataFrame(latent, index=rating_matrix.index,
                             columns=[f'PC{i+1}' for i in range(n_comp)])
    
    # PCA STEP 5: Reconstruct missing ratings
    # r̂_i = mean + t_i · p^T
    reconstructed = latent @ pc.T + item_means.values
    
    # Prediction for target item (first column)
    predictions = pd.Series(reconstructed[:, 0], index=rating_matrix.index).clip(1, 5)
    return predictions, eigenvalues, eigenvectors, n_comp, items, latent_df

In [44]:
# I1 with 5 peers
pred_I1_5, eigenvals_I1, eigenvecs_I1, n_comp_I1, items_I1, latent_I1 = pca_predict_with_steps(
    rating_matrix, I1, top5_I1
)

print(f"PCA RESULTS FOR I1 (movieId={I1}) WITH 5 PEERS")
print(f"\nItems in submatrix: {items_I1}")

print(f"\nEIGENVALUES (λ):")
print(f"Values: {eigenvals_I1.round(4)}")
print(f"Variance %: {(eigenvals_I1/eigenvals_I1.sum()*100).round(1)}")
print(f"Cumulative %: {(np.cumsum(eigenvals_I1)/eigenvals_I1.sum()*100).round(1)}")

print(f"\nEIGENVECTORS (Principal Components p):")
eigenvec_df = pd.DataFrame(eigenvecs_I1, index=items_I1,
                           columns=[f'PC{i+1}' for i in range(len(items_I1))])
print(eigenvec_df.round(4))

print(f"\nComponents selected: {n_comp_I1} (for 90% variance threshold)")
print(f"First PC (p1): {eigenvecs_I1[:, 0].round(4)}")

print(f"\nLATENT FACTORS (t_i = x_i · p), First 5 Users")
print(latent_I1.head(5).round(4))

PCA RESULTS FOR I1 (movieId=2) WITH 5 PEERS

Items in submatrix: [2, 26887, 106, 6956, 1922, 6177]

EIGENVALUES (λ):
Values: [0.0644 0.0025 0.0016 0.0016 0.0005 0.0004]
Variance %: [90.8  3.5  2.2  2.2  0.7  0.6]
Cumulative %: [ 90.8  94.3  96.5  98.7  99.4 100. ]

EIGENVECTORS (Principal Components p):
          PC1     PC2     PC3     PC4     PC5     PC6
2      1.0000  0.0033  0.0031 -0.0037 -0.0027  0.0013
26887  0.0040  0.0009 -0.9545  0.2982  0.0006 -0.0003
106    0.0023 -0.9584 -0.0761 -0.2405  0.0010  0.1333
6956   0.0016 -0.1218 -0.0177 -0.0573  0.0413 -0.9899
1922   0.0032 -0.2579  0.2873  0.9205 -0.0520 -0.0288
6177   0.0028 -0.0074  0.0164  0.0504  0.9978  0.0394

Components selected: 1 (for 90% variance threshold)
First PC (p1): [1.     0.004  0.0023 0.0016 0.0032 0.0028]

LATENT FACTORS (t_i = x_i · p), First 5 Users
           PC1
userId        
11      0.0008
12      0.0000
21      0.0000
33      0.0000
34     -0.2498


In [45]:
# I2 with 5 peers
pred_I2_5, eigenvals_I2, eigenvecs_I2, n_comp_I2, items_I2, latent_I2 = pca_predict_with_steps(
    rating_matrix, I2, top5_I2
)

print(f"PCA RESULTS FOR I2 (movieId={I2}) WITH 5 PEERS")
print(f"\nEIGENVALUES (λ):")
print(f"Values: {eigenvals_I2.round(4)}")
print(f"Variance %: {(eigenvals_I2/eigenvals_I2.sum()*100).round(1)}")

print(f"\n:EIGENVECTORS (p):")
eigenvec_df_I2 = pd.DataFrame(eigenvecs_I2, index=items_I2,
                              columns=[f'PC{i+1}' for i in range(len(items_I2))])
print(eigenvec_df_I2.round(4))

print(f"\nComponents selected: {n_comp_I2}")

print(f"\nLATENT FACTORS (First 5 Users)")
print(latent_I2.head(5).round(4))

PCA RESULTS FOR I2 (movieId=8860) WITH 5 PEERS

EIGENVALUES (λ):
Values: [0.0079 0.0049 0.0024 0.0016 0.0009 0.0007]
Variance %: [43.  26.7 13.   8.5  4.9  3.9]

:EIGENVECTORS (p):
           PC1     PC2     PC3     PC4     PC5     PC6
8860   -0.1054  0.9777 -0.1242  0.0685  0.1074  0.0374
6978   -0.0035  0.1225  0.9565 -0.0344 -0.0782  0.2506
581    -0.9932 -0.1096  0.0105 -0.0056  0.0359  0.0099
8650   -0.0164  0.0703  0.2410  0.0029 -0.0422 -0.9669
103688 -0.0039  0.0681 -0.0457 -0.9958 -0.0388 -0.0076
5815   -0.0464  0.0868 -0.0973  0.0489 -0.9888  0.0262

Components selected: 4

LATENT FACTORS (First 5 Users)
        PC1  PC2  PC3  PC4
userId                    
11      0.0  0.0  0.0  0.0
12      0.0  0.0  0.0  0.0
21      0.0  0.0  0.0  0.0
33      0.0  0.0  0.0  0.0
34      0.0  0.0  0.0  0.0


---
## Points 5-6: PCA with 10 Peers

In [33]:
pred_I1_10, eigenvals_I1_10, _, n_comp_I1_10, _, latent_I1_10 = pca_predict_with_steps(
    rating_matrix, I1, top10_I1
)
pred_I2_10, eigenvals_I2_10, _, n_comp_I2_10, _, latent_I2_10 = pca_predict_with_steps(
    rating_matrix, I2, top10_I2
)
print(f"10 Peers - I1: {n_comp_I1_10} components")
print(f"10 Peers - I2: {n_comp_I2_10} components")

10 Peers - I1: 4 components
10 Peers - I2: 8 components


---
## Latent Factors Comparison: 5 vs 10 Peers

In [46]:
print("LATENT FACTORS (First 10 Users)")
print(f"\nI1 with 5 PEERS")
print(latent_I1.head(10).round(4))
print(f"\nI1 with 10 PEERS")
print(latent_I1_10.head(10).round(4))

LATENT FACTORS (First 10 Users)

I1 with 5 PEERS
           PC1
userId        
11      0.0008
12      0.0000
21      0.0000
33      0.0000
34     -0.2498
40      0.0000
41      0.0000
46      0.0000
56      0.0000
66      0.0000

I1 with 10 PEERS
           PC1     PC2     PC3     PC4
userId                                
11      0.0008  0.0002  0.0008 -0.1838
12      0.0000  0.0000  0.0000  0.0000
21      0.0000  0.0000  0.0000  0.0000
33      0.0000  0.0000  0.0000  0.0000
34     -0.2498 -0.0008 -0.0008 -0.0014
40      0.0000  0.0000  0.0000  0.0000
41      0.0000  0.0000  0.0000  0.0000
46      0.0000  0.0000  0.0000  0.0000
56      0.0000  0.0000  0.0000  0.0000
66      0.0000  0.0000  0.0000  0.0000


---
## Predicted Ratings for Missing Values

Show first 10 users who had MISSING ratings for the target item, and their predicted values.

In [47]:
print("PREDICTED RATINGS FOR ORIGINALLY MISSING VALUES")
# Users who had MISSING rating for I1
missing_I1 = rating_matrix[I1].isna()
users_missing_I1 = missing_I1[missing_I1].index[:10]  # First 10

print(f"\nI1 (movieId={I1})")
print(f"Total users with missing I1 rating: {missing_I1.sum():,}")
print(f"\nFirst 10 users and their PREDICTED ratings:")
print("\n5 PEERS predictions:")
for user in users_missing_I1:
    print(f"  User {user}: predicted = {pred_I1_5[user]:.3f}")

print("\n10 PEERS predictions:")
for user in users_missing_I1:
    print(f"  User {user}: predicted = {pred_I1_10[user]:.3f}")

# Users who had MISSING rating for I2
missing_I2 = rating_matrix[I2].isna()
users_missing_I2 = missing_I2[missing_I2].index[:10]

print(f"\nI2 (movieId={I2})")
print(f"Total users with missing I2 rating: {missing_I2.sum():,}")
print(f"\nFirst 10 users and their PREDICTED ratings:")
print("\n5 PEERS predictions:")
for user in users_missing_I2:
    print(f"  User {user}: predicted = {pred_I2_5[user]:.3f}")

PREDICTED RATINGS FOR ORIGINALLY MISSING VALUES

I1 (movieId=2)
Total users with missing I1 rating: 12,888

First 10 users and their PREDICTED ratings:

5 PEERS predictions:
  User 11: predicted = 3.473
  User 12: predicted = 3.472
  User 21: predicted = 3.472
  User 33: predicted = 3.472
  User 40: predicted = 3.472
  User 41: predicted = 3.472
  User 46: predicted = 3.472
  User 56: predicted = 3.472
  User 66: predicted = 3.472
  User 71: predicted = 3.472

10 PEERS predictions:
  User 11: predicted = 3.472
  User 12: predicted = 3.472
  User 21: predicted = 3.472
  User 33: predicted = 3.472
  User 40: predicted = 3.472
  User 41: predicted = 3.472
  User 46: predicted = 3.472
  User 56: predicted = 3.472
  User 66: predicted = 3.472
  User 71: predicted = 3.472

I2 (movieId=8860)
Total users with missing I2 rating: 14,100

First 10 users and their PREDICTED ratings:

5 PEERS predictions:
  User 11: predicted = 3.376
  User 12: predicted = 3.376
  User 21: predicted = 3.376
  User 

---
## Evaluation: MAE and RMSE

In [48]:
def compute_mae(preds, test_df, item):
    subset = test_df[test_df['movieId'] == item]
    errors = [abs(row['rating'] - preds[row['userId']]) 
              for _, row in subset.iterrows() if row['userId'] in preds.index]
    return np.mean(errors) if errors else np.nan

def compute_rmse(preds, test_df, item):
    subset = test_df[test_df['movieId'] == item]
    errors = [(row['rating'] - preds[row['userId']])**2 
              for _, row in subset.iterrows() if row['userId'] in preds.index]
    return np.sqrt(np.mean(errors)) if errors else np.nan

mae_I1_5 = compute_mae(pred_I1_5, test_df, I1)
rmse_I1_5 = compute_rmse(pred_I1_5, test_df, I1)
mae_I2_5 = compute_mae(pred_I2_5, test_df, I2)
rmse_I2_5 = compute_rmse(pred_I2_5, test_df, I2)
mae_I1_10 = compute_mae(pred_I1_10, test_df, I1)
rmse_I1_10 = compute_rmse(pred_I1_10, test_df, I1)
mae_I2_10 = compute_mae(pred_I2_10, test_df, I2)
rmse_I2_10 = compute_rmse(pred_I2_10, test_df, I2)

print("EVALUATION RESULTS")
print(f"5 Peers - I1: MAE={mae_I1_5:.4f}, RMSE={rmse_I1_5:.4f}")
print(f"5 Peers - I2: MAE={mae_I2_5:.4f}, RMSE={rmse_I2_5:.4f}")
print(f"10 Peers - I1: MAE={mae_I1_10:.4f}, RMSE={rmse_I1_10:.4f}")
print(f"10 Peers - I2: MAE={mae_I2_10:.4f}, RMSE={rmse_I2_10:.4f}")

EVALUATION RESULTS
5 Peers - I1: MAE=0.6657, RMSE=0.8517
5 Peers - I2: MAE=0.5945, RMSE=0.7370
10 Peers - I1: MAE=0.6657, RMSE=0.8518
10 Peers - I2: MAE=0.5945, RMSE=0.7368


---
## Point 7: Compare 5 vs 10 Peers

In [49]:
comparison = pd.DataFrame({
    'Item': ['I1', 'I1', 'I2', 'I2'],
    'Peers': [5, 10, 5, 10],
    'Components': [n_comp_I1, n_comp_I1_10, n_comp_I2, n_comp_I2_10],
    'MAE': [mae_I1_5, mae_I1_10, mae_I2_5, mae_I2_10],
    'RMSE': [rmse_I1_5, rmse_I1_10, rmse_I2_5, rmse_I2_10]
})
print(comparison.round(4).to_string(index=False))

Item  Peers  Components    MAE   RMSE
  I1      5           1 0.6657 0.8517
  I1     10           4 0.6657 0.8518
  I2      5           4 0.5945 0.7370
  I2     10           8 0.5945 0.7368


---
## Points 8-9: Part 1 vs Part 2

In [38]:
print("Part 1 (Mean-Filling): predictions ~2.916 (uniform)")
print(f"Part 2 (MLE): I1 mean={pred_I1_5[missing_I1].mean():.3f}, std={pred_I1_5[missing_I1].std():.3f}")
print(f"Part 2 (MLE): I2 mean={pred_I2_5[missing_I2].mean():.3f}, std={pred_I2_5[missing_I2].std():.3f}")

Part 1 (Mean-Filling): predictions ~2.916 (uniform)
Part 2 (MLE): I1 mean=3.472, std=0.000
Part 2 (MLE): I2 mean=3.376, std=0.003
